In [ ]:
import pickle
from encode_utils.rerank_data import rerank_dist, rerank_single
from encode_utils.efficient_rerank import get_effrerank_model, run_comstyle
from encode_utils.sco_funct import weightaddprob, default_scofunct
from encode_utils.mt_scores import get_scores_auto
from encode_utils.new_flatten_lattice import get_dictlist
from encode_utils.new_mask_utils import randomsingle, useall
from transformers import AutoTokenizer
import pandas as pd
import numpy as np
import torch
import random
from generate_tables import metrics_mapping
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

In [2]:
# ISSUE #5 - Tokenization Splitting not Happening Well With X-Sum 

In [3]:
xsumcands = pd.read_csv("outputs/score_csvs/nounxsumbeam12v2.csv")

In [58]:
xsumcands['hyp'][500]

'Deutsche Bank shares fell as much as 5% after reports the bank has failed to reach an agreement with US authorities over a $14bn (£11bn) fine.'

In [59]:
# TODO do this for everything
mtb12 = pd.read_csv("outputs/predcsvs/noun_comstyle_xsum_randsing_8shot.csv", index_col=0)

In [69]:
testref = mtb12['ref'][22]

In [75]:
mtb12['hyp'][22]

"Plymouth Ar gyle midfielder Robbie Donald son has compared their League Two promotion push to Newcastle United's 2011 title success."

In [63]:
xlm_tok = AutoTokenizer.from_pretrained("xlm-roberta-base")
bartsum_tok = AutoTokenizer.from_pretrained("facebook/bart-large-xsum")

In [70]:
minis = bartsum_tok(testref).input_ids

In [72]:
testref

'Plymouth Argyle midfielder Ryan Donaldson says League Two teams see the Pilgrims as a major scalp.'

In [93]:
toks = [[m] for m in minis]
words = []
for i in range(1, len(toks)):
    dec = bartsum_tok.decode(toks[i-1]+ toks[i]).strip()
    #
    if " " in dec or "s>" in dec:
        print(dec)
        words.append(bartsum_tok.decode(toks[i]))
    else:
        words[-1] = dec
        toks[i] = toks[i-1]+ toks[i] 

<s>P
Plymouth Ar
Argyle midfielder
midfielder Ryan
Ryan Donald
Donaldson says
says League
League Two
Two teams
teams see
see the
the Pil
Pilgrims as
as a
a major
major scalp
scalp.</s>


In [99]:
xtokens

[[436, 538, 111443],
 [1172, 3432, 133],
 [4122, 28394, 56],
 [78201],
 [16692, 1681],
 [17378],
 [19175],
 [32964],
 [87199],
 [1957],
 [70],
 [13289, 113240, 7],
 [237],
 [10],
 [13036],
 [117906, 254, 5],
 [2]]

In [101]:
for x in xtokens:
    print([xlm_tok.decode(l) for l in x])

['P', 'ly', 'mouth']
['Ar', 'gy', 'le']
['mid', 'field', 'er']
['Ryan']
['Donald', 'son']
['says']
['League']
['Two']
['teams']
['see']
['the']
['Pil', 'grim', 's']
['as']
['a']
['major']
['scal', 'p', '.']
['</s>']


In [98]:
xtokens = []
for w in words:
    xtokens.append(xlm_tok(w).input_ids[1:-1])

In [80]:
encodemod = get_effrerank_model("comstyle")
xlm_tok = AutoTokenizer.from_pretrained("xlm-roberta-base")
argsinp = {
    'tok':xlm_tok,
    'model':encodemod,
    'device':device
}

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Encoder model frozen.
Loading weights from /mnt/data1/prasann/latticegen/lattice-generation/COMET/lightning_logs/version_43/checkpoints/epoch=3-step=130000.ckpt.


freeze embeds


KeyboardInterrupt: 

In [3]:
# get token level scores from model, given hypothesis and input source
def get_hyp_sco(inphyp, inpsrc, args):
    tok = args['tok']
    dev = args['device']
    model = args['model']

    # calculate inputs
    tokens = tok(inphyp, return_tensors='pt', truncation=True).to(dev)
    tokens = tokens.input_ids
    positionids = None
    toked_inp = tok(inpsrc, return_tensors="pt").to(dev)
    # get causal mask
    tmpmask = torch.tril(torch.ones(len(tokens[0]), len(tokens[0]))).unsqueeze(0).to(dev)
    # run through model
    predout = model(toked_inp.input_ids, toked_inp.attention_mask, tokens, positionids, \
        tmpmask)
    return predout['score']


In [15]:
torch.sum(get_hyp_sco(inex['hyp'], inex['src'], argsinp))

tensor(0.6200, device='cuda:1', grad_fn=<SumBackward0>)

In [ ]:
# Issue #4 - there's a difference between original generation and new generation scores

In [9]:
inex = mtb12.loc[0]

In [4]:
tmpset = mtb12.loc[:3]
tmpset = tmpset.rename(columns={'dupcqe':"dco2"})

In [6]:
tmpset

,src,ref,hyp,dcqeold,comet,cqe,posthoc,dco2,dupcqe
0,"Comme pour tout compromis, les parties prenant...","As with any compromise, the contending parties...","As with any compromise, stakeholders would hav...",0.672990,0.776076,0.526303,0.390460,0.640510,0.640568
1,"Comme pour tout compromis, les parties prenant...","As with any compromise, the contending parties...","As with any compromise, stakeholders would hav...",0.664692,0.766729,0.548467,0.393152,0.635732,0.635743
2,"Comme pour tout compromis, les parties prenant...","As with any compromise, the contending parties...","As with any compromise, stakeholders would hav...",0.636959,0.759802,0.545454,0.405204,0.610568,0.610579
3,"Comme pour tout compromis, les parties prenant...","As with any compromise, the contending parties...","As with any compromise, the stakeholders would...",0.667578,0.771617,0.517116,0.406245,0.643053,0.643061


In [5]:
metrics_mapping("dupcqe", tmpset)

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Encoder model frozen.
Loading weights from /mnt/data1/prasann/latticegen/lattice-generation/COMET/lightning_logs/version_43/checkpoints/epoch=3-step=130000.ckpt.


freeze embeds


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
Predicting DataLoader 0: 100%|██████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  5.42it/s]


TOOK TIME  4.18


In [4]:
base = "outputs/graph_pickles/frtest_reversed/"
noun_explode = pd.read_csv("outputs/score_csvs/nounlargeexplodev1.csv")

In [7]:
smallset = noun_explode.loc[:3]

In [8]:
smallset['utnounold'] = smallset['utnoun']
smallset = smallset.drop(columns=['utnoun'])

/tmp/ipykernel_4129421/615870310.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smallset['utnounold'] = smallset['utnoun']


In [9]:
metrics_mapping('utnoun', smallset)

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Encoder model frozen.
Loading weights from /mnt/data1/prasann/latticegen/lattice-generation/COMET/lightning_logs/version_44/checkpoints/epoch=9-step=40000.ckpt.


freeze embeds


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
Predicting DataLoader 0: 100%|██████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.92it/s]

tensor([[28],
        [29],
        [28],
        [29]], device='cuda:1')


TOOK TIME  3.75


In [10]:
smallset

,Unnamed: 0,src,ref,hyp,unique_nouns,utnounold,utnoun
0,0,Après tout le Soudan est de la taille de l'Eur...,"After all, Sudan is the size of Western Europe...","After all, Sudan is the size of Western Europe...",7,0.702689,0.740255
1,1,Après tout le Soudan est de la taille de l'Eur...,"After all, Sudan is the size of Western Europe...","After all, the Sudan is the size of Western Eu...",7,0.721806,0.759443
2,2,Après tout le Soudan est de la taille de l'Eur...,"After all, Sudan is the size of Western Europe...","After all, Sudan is the size of Western Europe...",7,0.688667,0.726136
3,3,Après tout le Soudan est de la taille de l'Eur...,"After all, Sudan is the size of Western Europe...","After all, the Sudan is the size of Western Eu...",7,0.708893,0.746611


In [17]:
torch.sum(get_hyp_sco(smallset['hyp'][1]))

tensor(25, device='cuda:1')


tensor(0.7594, device='cuda:1', grad_fn=<SumBackward0>)

In [5]:
# Issue #3 - there's a difference between gold selected with standard re-rank and with EEL

In [6]:
# get token level scores from model, given hypothesis and input source
def get_hyp_sco(inphyp, inpsrc, args):
    tok = args['tok']
    dev = args['device']
    model = args['model']

    # calculate inputs
    tokens = tok(inphyp, return_tensors='pt', truncation=True).to(dev)
    tokens = tokens.input_ids
    positionids = None
    toked_inp = tok([inpsrc], return_tensors="pt").to(dev)
    # get causal mask
    tmpmask = torch.tril(torch.ones(len(tokens[0]), len(tokens[0]))).unsqueeze(0).to(dev)
    # run through model
    predout = model(toked_inp.input_ids, toked_inp.attention_mask, tokens, positionids, \
        tmpmask)
    return predout['score']

argsinp = {
    'tok':xlm_tok,
    'model':encodemod,
    'device':device
}

NameError: name 'xlm_tok' is not defined

In [4]:
# get token level scores from model
def get_hyp_sco(inphyp, posids=None):
    
    tokens = xlm_tok(inphyp, return_tensors='pt').to(device)
    tokens = tokens.input_ids
    if posids is None: 
        positionids = None
    else:
        # get token at the end
        positionids = torch.tensor(posids+[posids[-1]+1]).unsqueeze(0).to(device)
    tmpmask = torch.tril(torch.ones(len(tokens[0]), len(tokens[0]))).unsqueeze(0).to(device)

    toked_inp = xlm_tok(["noun"], return_tensors="pt").to(device)
    predout = encodemod(toked_inp.input_ids, toked_inp.attention_mask, tokens, positionids, \
        tmpmask)
    tmppred = predout['score']
    #norm = predout['norm']
    return tmppred

# TODO do a validation that old score generation way and current have same bests
def get_ind_result(ind):
    graph = pickle.load(open(base+str(ind), 'rb'))
    texplode = noun_explode[noun_explode['ref']==graph['ref']].reset_index()
    # recalculate noun scores for all
    cscos = []
    for t in list(texplode['hyp']):
        hs = get_hyp_sco(t)
        cscos.append(torch.sum(get_hyp_sco(t)))#*(hs.shape[1]-1))
    print("scodist - ", [float(f) for f in cscos])
    print("max - ", float(max(cscos)))
    bestpath , flattened, pnodes, mask, sents, posids, pred, _, \
            flnodes, dpath, beplist, besclist, totnodes, bsco = run_comstyle(graph, encodemod, default_scofunct, "noun", {'afunc':randomsingle}, True)
    predhyp = bestpath[0][4:]
    ph = get_hyp_sco(predhyp)
    predsco = torch.sum(ph)#*(ph.shape[1]-1)
    print("pred - ", float(predsco))
    print("predhyp")
    return mask, sents, posids, pred, list(texplode['hyp'])

In [ ]:
msk, snts, pids, scos, hyps = get_ind_result(18)

In [47]:
hyps[-2]

'To some extent, China is going to face obstacles on its way, and there is no guarantee that its leaders will admit their failures or adjust their strategy accordingly.'

In [8]:
posmat = pids.expand(len(snts[0]), len(snts[0])).to("cpu")
posver = posmat*msk
inpmat = snts.expand(len(snts[0]), len(snts[0])).to("cpu")
inpmat = inpmat*(msk>0)
scomat = scos.squeeze(-1).expand(len(snts[0]), len(snts[0])).to("cpu")
scomat = scomat*(msk>0)

In [33]:
pids

tensor([[ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
         20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37,
         10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27,
         28, 29, 30, 31, 32, 33, 34, 35, 36, 37,  4,  5,  6,  7,  8,  9, 10, 11,
         12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29,
         30, 31, 32, 33, 34, 35, 36, 37, 38, 11, 12, 13, 14, 15, 16, 17, 18, 19,
         20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37,
         38,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24,
         25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,  9, 10, 11, 12, 13, 14,
         15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32,
         33, 34, 35, 36, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28,
         29, 30, 31, 32, 33, 34, 35, 36,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12,
         13, 14, 15, 16, 17,

In [36]:
scomat[35][:37]*torch.max(pids).to("cpu")

tensor([0.0000, 0.0315, 0.0320, 0.0334, 0.3809, 0.0520, 0.9946, 0.2352, 0.2196,
        0.1561, 1.0815, 0.3335, 0.4159, 0.3077, 1.5724, 0.1403, 0.4810, 0.4012,
        0.4049, 0.5259, 2.2846, 0.6981, 0.4920, 2.6411, 0.7156, 0.7587, 0.6133,
        3.0344, 0.8132, 0.7931, 1.2603, 0.4955, 3.8419, 1.0233, 0.5695, 0.3316,
        0.0000])

In [44]:
(get_hyp_sco(hyps[11])*36).shape

torch.Size([1, 37, 1])

In [22]:
xlm_tok.decode(inpmat[35])

'<s> To a certain extent, China will encounter obstacles on its way, and there is no guarantee that its leaders will admit their failures or adjust their strategy accordingly.<s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s>

In [11]:
for i in range(len(posver)):
    for j in range(1, len(posver[0])):
        val = posver[i][j]
        if val == 0:
            continue
        if val <= posver[i][j-1]:
            print(i)

In [43]:
# mask has no recomb
assert torch.sum(msk!=torch.tril(msk))==0
# no weird posids

In [5]:
# Issue #2 - graph may be getting corrupted during the process of conversion, make sure identical exploded even after conversion
mb_tok = AutoTokenizer.from_pretrained("facebook/mbart-large-50-many-to-one-mmt")

# recursive method to sanity check first (assume no recomb, no cycles)
def explode_path(ind, processed=False):
    graph = pickle.load(open(base+str(ind), 'rb'))
    allpaths = []
    if processed:
        fldict, flnodes = get_dictlist(graph, True)
        explode_helper([], flnodes[0], allpaths, xlm_tok)
    else:
        explode_helper([], graph['root'], allpaths, mb_tok)
        allpaths = [a[10:] for a in allpaths]
    nexplode = noun_explode[noun_explode['ref']==graph['ref']].reset_index()
    return allpaths, list(nexplode['hyp'])

# helper for exploding paths
def explode_helper(prevpath, node, apaths, tok):
    prevpath.append(node.token_idx)
    if len(node.nextlist)==0:
        #print(prevpath)
        apaths.append(mb_tok.decode(prevpath))
    else:
        for n in node.nextlist:
            explode_helper(prevpath, n, apaths, tok)
    prevpath.pop()

In [ ]:
apths = []
explode_helper([], pnodes[0][0], apths, xlm_tok)

In [4]:
# Issue #1 - changing pos-ids with identical strings changes token scores for all things
# score check (at token level) the best option, and the new option (should be same until they get to non-identical tokens)

In [8]:
# get token level scores from model
def get_hyp_sco_verb(inphyp, posids=None):
    
    tokens = xlm_tok(inphyp, return_tensors='pt').to(device)
    tokens = tokens.input_ids

    if posids is None: 
        # have to add 2 because of how xlm roberta works
        positionids = torch.arange(len(tokens[0])).unsqueeze(0).to(device)+2
        print(positionids)

    elif -1 in posids:
        positionids = None
    else:
        # get token at the end
        positionids = posids
        #torch.tensor(posids+[posids[-1]+1]).unsqueeze(0).to(device)
        print(positionids)

    # get a causal mask to use
    tmpmask = torch.tril(torch.ones(len(tokens[0]), len(tokens[0]))).unsqueeze(0).to(device)
    
    # this is a dummy string (using CQE model architecture which requires an "input" pass)
    toked_inp = xlm_tok(["noun"], return_tensors="pt").to(device)
    
    # run inputs through the model
    predout = encodemod(toked_inp.input_ids, toked_inp.attention_mask, tokens, positionids, \
        tmpmask)
    
    tmppred = predout['score']
    return tmppred, tokens, positionids, tmpmask


In [16]:
# get scores for identical strings, with different posids
sco_default = get_hyp_sco_verb("I am a bat")
modpos = sco_default[2]
# change last position id
modpos[0][4] = 7
sco_mod = get_hyp_sco_verb("I am a bat", modpos)

tensor([[2, 3, 4, 5, 6, 7]], device='cuda:1')
tensor([[2, 3, 4, 5, 7, 7]], device='cuda:1')


In [18]:
# these two are actually fine. There is some sort of issue in no-recomb cases
print(sco_default[0])
print(sco_mod[0])

tensor([[[0.0000],
         [0.0042],
         [0.0042],
         [0.0043],
         [0.0508],
         [0.0214]]], device='cuda:1', grad_fn=<DivBackward0>)
tensor([[[0.0000],
         [0.0042],
         [0.0042],
         [0.0043],
         [0.0557],
         [0.0211]]], device='cuda:1', grad_fn=<DivBackward0>)
